<a href="https://colab.research.google.com/github/Sofigam00s/Procesamiento-del-habla-IA/blob/main/TP4_chatbot_retrieval_template_GamalloSofia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP4 Chatbots

**Alumna: Gamallo Sofia Antonia**


In [ ]:
!pip install spacy --quiet
!python -m spacy download es_core_news_sm --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 80.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")
import es_core_news_sm
nlp = es_core_news_sm.load()
doc = nlp("Esto es una frase.")
print([(w.text, w.pos_) for w in doc])

[('Esto', 'PRON'), ('es', 'AUX'), ('una', 'DET'), ('frase', 'NOUN'), ('.', 'PUNCT')]


## Chatbots basados en recuperación

En inglés information retrieval chatbots

## Motor de búsqueda

* Búsqueda por palabras clave: Extrae palabras clave de la pregunta del usuario y busca coincidencias en las preguntas almacenadas.

* Similitud del coseno: Si has representado las preguntas como vectores (por ejemplo, usando TF-IDF o word embeddings), puedes usar la similitud del coseno para medir la distancia entre las preguntas.

* Word embeddings: Utiliza modelos de word embeddings como Word2Vec o BERT para obtener representaciones semánticas de las preguntas y las consultas del usuario.

### Búsqueda por palabras claves

In [ ]:
tu_diccionario = {
   "hola": "¡Hola! ¿En qué puedo ayudarte?",
   "adiós": "Hasta luego. ¡Que tengas un buen día!",
   "información": "¿Qué tipo de información estás buscando?",
   # Agrega más entradas de diccionario según tus necesidades
}


In [ ]:
def responder_pregunta(pregunta):
    pregunta_procesada = nlp(pregunta.lower())  # Procesa la pregunta y convierte a minúsculas
    respuesta = "Lo siento, no entiendo tu pregunta."

    # Busca una coincidencia en el diccionario
    for palabra in pregunta_procesada:
        # regresa la primer coincidencia que encuentra
        if palabra.text in tu_diccionario:
            respuesta = tu_diccionario[palabra.text]
            break

    return respuesta


In [ ]:
'''while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot: Hasta luego.")
        break
    respuesta = responder_pregunta(entrada_usuario)
    print("Chatbot:", respuesta)'''


'while True:\n    entrada_usuario = input("Tú: ")\n    if entrada_usuario.lower() == "salir":\n        print("Chatbot: Hasta luego.")\n        break\n    respuesta = responder_pregunta(entrada_usuario)\n    print("Chatbot:", respuesta)'

## Búsqueda por similitud

Para los chatbots basados ​​en recuperación, es común utilizar bolsas de palabras (bag of words) o tf-idf para calcular la similitud de intenciones.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Datos de ejemplo
preguntas = ["¿Qué es el aprendizaje automático?",
             "¿Cómo funciona la regresión lineal?"]
respuestas = ["El aprendizaje automático es una rama de la inteligencia artificial...",
              "La regresión lineal es un método de modelado..."]

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preguntas)

# Función para encontrar la mejor coincidencia
def responder_pregunta(consulta_usuario):
    consulta_vec = vectorizer.transform([consulta_usuario])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    print(similitudes)
    indice_mejor_coincidencia = similitudes.argmax()
    print(indice_mejor_coincidencia)
    return respuestas[indice_mejor_coincidencia]


In [ ]:

# Ejemplo de consulta
consulta = "¿Qué es la regresión lineal?"
print(responder_pregunta(consulta))


[0.4 0.6]
1
La regresión lineal es un método de modelado...


## Búsqueda por similitud en embeddings

Puedes vectorizar el texto usando embeddings, como vimos la clase pasada.


# Actividades

### 1) Elaborar un dataset de preguntas y respuestas para crear un Chatbot para un aplicación particular. ( 3 puntos )

1.1 Debe definir la aplicación (atención al cliente bancario, atención a estudiantes universitarios, etc).
1.2 El listado de preguntas y respuestas debe tener como mínimo 20 elementos pregunta - respuesta.

###  2) Crear el chatbot utilizando TFIDF y similitud del coseno. (1 punto)

### 3) Crear otro chatbot utilizando embeddings. Indique cuál embedding (1 punto) pre-entrenado eligió.

### 4) Muestra ambos chatbots funcionando (1 punto)

Adjuntar la lista de preguntas utilizadas para probar el funcionamiento.

### 5) Añade tus conclusiones de todo lo realizado (2 punto)

### 6) BONUS: usa lo realizado en 1 y 3 para crear un chatbot RAG. (2 puntos)

* Utiliza un modelo LLM pre-entrenado.

* Este punto no es obligatorio de realizar para quienes quieran regularizar / recuperar y luego rendirán en mesa.
* Para quienes tienen condiciones para promocionar (han realizado y entregado los TPs a tiempo) la resolución de este ejercicio será tenida en cuenta para sumar a la promoción.

### 7) No olvides:

* Explicar tus decisiones y configuraciones. Añadir tus conclusiones.
* Anunciar en el foro cuál será tu aplicación y postear tu entrega y tus avances.
* Debes subir tu notebook a un repo GitHub público de tu propiedad compartido + enlace colab.
* Documentar todo el proceso.





## 1. Dataset sobre un chatbot de tienda de ropa online

Para este trabajo voy a relizar un chatbot de una pagina de tienda de ropa online, que pueda responder a las dudas más frecuentes que se tienen en este rubro, como la forma de compra, metodos de pago, seguimiento de envío, etc.

* Importar las librerias necesarias

In [ ]:
!pip install sentence-transformers
!pip install sentence-transformers openai

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import openai
import re
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

* Cargar el dataset que subí al repositorio de GitHub en la carpeta del tp4

In [ ]:
# URL del dataset (Raw content)
# Cambiar la URL para apuntar al contenido raw del archivo CSV en GitHub
# This URL provides the raw data, not the GitHub page HTML
dataset_url = 'https://raw.githubusercontent.com/Sofigam00s/Procesamiento-del-habla-IA/main/TP4_Chatbots/Chatbot_tienda_de%20_ropa_online.csv'

# Importar el dataset
# Add error handling for bad lines
df = pd.read_csv(dataset_url, sep=',', on_bad_lines='skip')
df

,Pregunta,Respuesta
0,Hola,Hola! En que puedo ayudarte?
1,¿Cómo realizo una compra?,"Seleccioná los productos que quieras, agregalo..."
2,¿Cómo creo una cuenta?,"Entrá en “Crear cuenta”, completá tus datos y ..."
3,No me llegó el mail de confirmación de cuenta.,"Si aún no llegó el mail de confirmación, debés..."
4,¿Cuál es el mínimo de compra?,"No contamos con mínimo de compra, podés compra..."
5,¿Los precios incluyen IVA?,"Sí, todos los precios publicados ya tienen el ..."
6,¿Los productos tienen cambio?,"Sí, podés cambiar el producto siempre que esté..."
7,¿Puedo cambiar un producto si ya hice el pedido?,No se pueden modificar pedidos una vez confirm...
8,¿Cuándo vuelve a ingresar el stock?,Si querés saber cuando ingresa el sotck de un ...
9,No me deja agregar el mail.,"Si ocurre un error con el ingreso de mail, pro..."


### 2. Crear el chatbot con matriz TF-IDF y similitud del coseno

* **Preprocesar el texto**

  Para poder aplicar la matriz TF-IDF y luego los embedding, primero tengo que hacer un preprocesamiento. En este caso particular elegí un tema sobre tienda de ropa online, por lo que el chatbot trabajará con textos que considero simples, tanto en las preguntas como las respuestas, y además el vocabulario será muy similar. Por lo tanto, solo voy a limpiar el texto de manera muy básica para mantener la exactitud del chatbot. Sin embargo, en caso de haber tenido preguntas y respuestas más complejas, sí hubiese utilizado un preprocesamiento más profundo que involucre la lemmatización o eliminación de stop_words.

In [ ]:
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = texto.lower()
    texto = re.sub(r'[^\w\s¡!¿?]', '', texto)

    # Remover espacios extra
    texto = ' '.join(texto.split())

    return texto

# Aplicar limpieza
df['pregunta_limpia'] = df['Pregunta'].apply(limpiar_texto)
df['respuesta_limpia'] = df['Respuesta'].apply(limpiar_texto)

* **Crear el chatbot con la matriz TF-IDF**

In [ ]:
class ChatbotTFIDF:
    def __init__(self, preguntas, respuestas):
        self.preguntas = preguntas
        self.respuestas = respuestas
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(preguntas)

    def responder(self, consulta_usuario, umbral=0.6):
        consulta_limpia = limpiar_texto(consulta_usuario)
        consulta_vec = self.vectorizer.transform([consulta_limpia])

        similitudes = cosine_similarity(consulta_vec, self.tfidf_matrix).flatten()
        indice_mejor = similitudes.argmax()
        mejor_similitud = similitudes[indice_mejor]

        if mejor_similitud < umbral:
            return "Lo siento, no entiendo tu pregunta. ¿Podrías reformularla?", mejor_similitud

        return self.respuestas[indice_mejor], mejor_similitud


### 3. Chatbot con embedding

Para este chatbot voy a utilizar el modelo de BERT, porque es de lo más conocidos y utilizados.

In [ ]:
class ChatbotEmbeddings:
    def __init__(self, preguntas, respuestas, modelo='paraphrase-multilingual-MiniLM-L12-v2'):
        self.preguntas = preguntas
        self.respuestas = respuestas
        self.model = SentenceTransformer(modelo)
        # Encode questions and move the resulting tensor to the CPU
        self.embedding_matrix = self.model.encode(preguntas, convert_to_tensor=True).cpu()

    def responder(self, consulta_usuario, umbral=0.6):
        # Encode the user query and move the resulting tensor to the CPU
        consulta_embedding = self.model.encode([consulta_usuario], convert_to_tensor=True).cpu()
        # Now both tensors are on the CPU, cosine_similarity can be calculated
        similitudes = cosine_similarity(consulta_embedding, self.embedding_matrix)[0]

        indice_mejor = np.argmax(similitudes)
        mejor_similitud = similitudes[indice_mejor]

        if mejor_similitud < umbral:
            return "Lo siento, no entiendo tu pregunta. ¿Podrías reformularla?", float(mejor_similitud)

        return self.respuestas[indice_mejor], float(mejor_similitud)

### 4. Mostrar funcionamiento de los chatbots

Para esta parte voy hacer 5 preguntas al azar de mi dataset y comprar la similitud de respuesta en cada chatbot

In [ ]:
# Mostrar 5 respuestas al azar del dataset
preguntas_aleatorias = df[['Pregunta', 'Respuesta']].sample(5)
preguntas_aleatorias

,Pregunta,Respuesta
28,¿Tienen tienda física?,"No, por el momento solo vendemos online. Todos..."
29,¿Puedo retirar el envío en la tienda?,Actualmente no contamos con retiro en tienda. ...
34,¿Cómo me contacto con un asesor?,"Podés escribirnos por WhatsApp, chat en vivo o..."
15,¿Puedo pagar en cuotas?,"Sí, si tu tarjeta lo permite, podés pagar en c..."
21,¿Cuánto tarda mi envío?,El tiempo estimado de entrega es entre 7 y 40 ...


Me salieron las siguientes preguntas al azar:
1. ¿Los precios incluyen IVA? 	Sí, todos los precios publicados ya tienen el IVA incluido.
2. ¿Cómo cancelo una compra? Si el pedido aún no fue despachado, podés cancelarlo desde “Mis compras” seleccionando la opción “Cancelar compra”.
3. No me llegó el seguimiento. Si no lo recibiste, escribinos por el chat o al mail de atención al cliente y te lo volvemos a enviar.
4. ¿Cómo creo una cuenta? Entrá en “Crear cuenta”, completá tus datos y listo. Solo necesitás un mail válido.
5. ¿Hacen envíos? Sí, realizamos envíos a todo el país con Correo Argentino.

Voy a probar el chatbot escribiendo las preguntas simples.


In [ ]:
# prompt: mostrar a chatbottfidf y chatbotembeddings funcionando

# Instantiate and test the TFIDF chatbot
chatbot_tfidf = ChatbotTFIDF(df['pregunta_limpia'].tolist(), df['Respuesta'].tolist())

print("--- Chatbot TF-IDF ---")
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot TF-IDF: Hasta luego.")
        break
    respuesta, similitud = chatbot_tfidf.responder(entrada_usuario)
    print(f"Chatbot TF-IDF: {respuesta} (Similitud: {similitud:.4f})")


--- Chatbot TF-IDF ---
Tú: hola
Chatbot TF-IDF: Hola! En que puedo ayudarte? (Similitud: 1.0000)
Tú: los precios incluyen iva
Chatbot TF-IDF: Sí, todos los precios publicados ya tienen el IVA incluido. (Similitud: 1.0000)
Tú: como creo una cunta
Chatbot TF-IDF: Entrá en “Crear cuenta”, completá tus datos y listo. Solo necesitás un mail válido. (Similitud: 0.7775)
Tú: como cancelo una compra
Chatbot TF-IDF: Si el pedido aún no fue despachado, podés cancelarlo desde “Mis compras” seleccionando la opción “Cancelar compra”. (Similitud: 0.9349)
Tú: no me llego el seguimiento
Chatbot TF-IDF: Si no lo recibiste, escribinos por el chat o al mail de atención al cliente y te lo volvemos a enviar. (Similitud: 0.6945)
Tú: hacen envios?
Chatbot TF-IDF: Sí, realizamos envíos a todo el país con Correo Argentino. (Similitud: 0.7071)
Tú: salir
Chatbot TF-IDF: Hasta luego.


In [ ]:
# Instantiate and test the Embeddings chatbot
chatbot_embeddings = ChatbotEmbeddings(df['Pregunta'].tolist(), df['Respuesta'].tolist())

print("\n--- Chatbot Embeddings ---")
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot Embeddings: Hasta luego.")
        break
    respuesta, similitud = chatbot_embeddings.responder(entrada_usuario)
    print(f"Chatbot Embeddings: {respuesta} (Similitud: {similitud:.4f})")


--- Chatbot Embeddings ---
Tú: hola
Chatbot Embeddings: Hola! En que puedo ayudarte? (Similitud: 0.9834)
Tú: los precios incluyen iva
Chatbot Embeddings: No encontré información sobre eso. ¿Podés reformular? (Similitud: 0.5594)
Tú: como cancelo una compra
Chatbot Embeddings: Si el pedido aún no fue despachado, podés cancelarlo desde “Mis compras” seleccionando la opción “Cancelar compra”. (Similitud: 0.9255)
Tú: no me llego el seguimiento
Chatbot Embeddings: Si no lo recibiste, escribinos por el chat o al mail de atención al cliente y te lo volvemos a enviar. (Similitud: 0.6132)
Tú: como creo una cuenta
Chatbot Embeddings: Entrá en “Crear cuenta”, completá tus datos y listo. Solo necesitás un mail válido. (Similitud: 0.9305)
Tú: hacen envios?
Chatbot Embeddings: Sí, realizamos envíos a todo el país con Correo Argentino. (Similitud: 0.7814)
Tú: salir
Chatbot Embeddings: Hasta luego.


### Resultados:

Los resultados de similitud de respuesta para cada chatbot fueron:

0. TF-IDF = 1.0 | Embeddings = 0.98
1. TF-IDF = 1.0 | Embeddings = 0.55 (no supo responder)
2. TF.IDF = 0.93 | Embeddings = 0.92
3. TF-IDF = 0.69 | Embeddings = 0.61
4. IF-IDF = 0.93 | Embeddings = 0.93
5. TF-IDF = 0.70 | Embedings = 0.78



El 0. corresponde al saludo inicial: Hola.

Todas las preguntas fueron respondidas con su correcta respuesta, exceptuando la pregunta 2 donde el chatbot de embeddings no supo responder si los precios incluían IVA.

### **EXTRA:** verificar que una limpieza mayor empeora el modelo

Para verificar el modelo de los chtbots empeoran con una lmpieza mas avanzada voy a probar rapidamente el funcionamiento para poder compararlos.

* Limpieza completa

In [ ]:
nlp_spacy = spacy.load("es_core_news_sm")

def limpiar_texto_avanzado(texto):
    if pd.isna(texto):
        return ""
    texto = texto.lower()
    texto = re.sub(r'[^\w\s¡!¿?]', '', texto)
    texto = ' '.join(texto.split())
    doc = nlp_spacy(texto)

    # Eliminar stopwords y lematizar
    tokens_limpios = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]

    return ' '.join(tokens_limpios)

# Aplicar limpieza avanzada
df['pregunta_limpia_avanzada'] = df['Pregunta'].apply(limpiar_texto_avanzado)
df['respuesta_limpia_avanzada'] = df['Respuesta'].apply(limpiar_texto_avanzado)

In [ ]:
chatbot_tfidf_avanzado = ChatbotTFIDF(df['pregunta_limpia_avanzada'].tolist(), df['Respuesta'].tolist())

print("--- Chatbot TF-IDF (Limpieza Avanzada) ---")
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot TF-IDF (Limpieza Avanzada): Hasta luego.")
        break
    respuesta, similitud = chatbot_tfidf_avanzado.responder(entrada_usuario)
    print(f"Chatbot TF-IDF (Limpieza Avanzada): {respuesta} (Similitud: {similitud:.4f})")

--- Chatbot TF-IDF (Limpieza Avanzada) ---
Tú: hola
Chatbot TF-IDF (Limpieza Avanzada): Hola! En que puedo ayudarte? (Similitud: 1.0000)
Tú: los precios incluyen iva
Chatbot TF-IDF (Limpieza Avanzada): Lo siento, no entiendo tu pregunta. ¿Podrías reformularla? (Similitud: 0.5774)
Tú: como cancelo una compra
Chatbot TF-IDF (Limpieza Avanzada): Seleccioná los productos que quieras, agregalos al carrito y hacé clic en “Finalizar compra”. Después seguí los pasos del sistema para el pago y la dirección. (Similitud: 0.6349)
Tú: no me llego el seguimiento
Chatbot TF-IDF (Limpieza Avanzada): Si no lo recibiste, escribinos por el chat o al mail de atención al cliente y te lo volvemos a enviar. (Similitud: 0.7071)
Tú: como creo una cuenta
Chatbot TF-IDF (Limpieza Avanzada): Lo siento, no entiendo tu pregunta. ¿Podrías reformularla? (Similitud: 0.0000)
Tú: hacen envios?
Chatbot TF-IDF (Limpieza Avanzada): Lo siento, no entiendo tu pregunta. ¿Podrías reformularla? (Similitud: 0.0000)
Tú: salir
Cha

In [ ]:
# Instantiate and test the Embeddings chatbot with advanced cleaning
chatbot_embeddings_avanzado = ChatbotEmbeddings(df['pregunta_limpia_avanzada'].tolist(), df['Respuesta'].tolist())

print("\n--- Chatbot Embeddings (Limpieza Avanzada) ---")
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot Embeddings (Limpieza Avanzada): Hasta luego.")
        break
    respuesta, similitud = chatbot_embeddings_avanzado.responder(entrada_usuario)
    print(f"Chatbot Embeddings (Limpieza Avanzada): {respuesta} (Similitud: {similitud:.4f})")


--- Chatbot Embeddings (Limpieza Avanzada) ---
Tú: hola
Chatbot Embeddings (Limpieza Avanzada): Hola! En que puedo ayudarte? (Similitud: 1.0000)
Tú: los precios incluyen iba
Chatbot Embeddings (Limpieza Avanzada): Sí, todos los precios publicados ya tienen el IVA incluido. (Similitud: 0.7349)
Tú: como cancelo una compra
Chatbot Embeddings (Limpieza Avanzada): Si el pedido aún no fue despachado, podés cancelarlo desde “Mis compras” seleccionando la opción “Cancelar compra”. (Similitud: 0.9539)
Tú: no me llego el seguimiento
Chatbot Embeddings (Limpieza Avanzada): No encontré información sobre eso. ¿Podés reformular? (Similitud: 0.4305)
Tú: como creo una cuenta
Chatbot Embeddings (Limpieza Avanzada): No encontré información sobre eso. ¿Podés reformular? (Similitud: 0.5084)
Tú: hacen envios?
Chatbot Embeddings (Limpieza Avanzada): Puede haber demoras por logística externa. Revisá el seguimiento desde “Mis compras” o con el código que te enviamos por mail. (Similitud: 0.7441)
Tú: salir
Ch

### Resultados

0. TF-IDF LA = 1.0 | Embbedings LA = 1.0
1. TF-IDF LA = 0.57 (no supo responder) | Embeddings LA = 0.73
2. TF-IDF LA = 0.63 (respuesta incorrecta) | Embeddings LA = 0.95
3. TF-IDF LA = 0.70 | Embeddings LA = 0.43 (no supo responder)
4. TF-IDF LA = 0.0 (no supo responder) | Embeedings LA = 0.50 (no supo responder)
5. TF-IDF LA = 0.0 (no supo responder) | Embeddings LA = 0.74 (fue una respuesta incorrecta)

Con la limpieza avanzada, los chatbot no responden correctamente o no saben la respuesta, aun teniendo una similitud moderada o alta.

### 6. Crear un chatbot RAG

In [ ]:
def preparar_chatbot_rag(preguntas, respuestas, modelo_embed='paraphrase-multilingual-MiniLM-L12-v2', modelo_llm='google/flan-t5-small'):
    from sentence_transformers import SentenceTransformer
    from transformers import pipeline

    embedder = SentenceTransformer(modelo_embed)
    preguntas_embed = embedder.encode(preguntas, convert_to_tensor=True).cpu()

    try:
        generator = pipeline("text2text-generation", model=modelo_llm)
    except:
        generator = None
        print("Generador no disponible. Se usará la respuesta original.")

    return preguntas, respuestas, preguntas_embed, embedder, generator

def responder_rag(consulta, preguntas, respuestas, preguntas_embed, embedder, generator, umbral=0.6):
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    consulta_embed = embedder.encode([consulta], convert_to_tensor=True).cpu()
    similitudes = cosine_similarity(consulta_embed, preguntas_embed)[0]

    idx = np.argmax(similitudes)
    score = float(similitudes[idx])
    respuesta_base = respuestas[idx]

    if score < umbral or generator is None:
        return respuesta_base, score

    prompt = f"Mejora esta respuesta de atención al cliente: {respuesta_base}"

    try:
        respuesta_gen = generator(prompt, max_new_tokens=50, do_sample=False)[0]['generated_text'].strip()
        if len(respuesta_gen) > 10 and "tienda" in respuesta_gen.lower():
            return respuesta_gen, score
    except:
        pass

    return respuesta_base, score

In [ ]:
# Preparar chatbot
preguntas = df['Pregunta'].tolist()
respuestas = df['Respuesta'].tolist()
preguntas, respuestas, preguntas_embed, embedder, generator = preparar_chatbot_rag(preguntas, respuestas)

# Chat interactivo
print("\n--- Chatbot RAG ---")
while True:
    consulta = input("Tú: ")
    if consulta.lower() == "salir":
        print("Chatbot RAG: Hasta luego.")
        break
    respuesta, similitud = responder_rag(consulta, preguntas, respuestas, preguntas_embed, embedder, generator)
    print("-" * 50)
    print(f"Chatbot RAG: {respuesta}")
    print(f"Similitud: {similitud:.4f}")
    print("-" * 50)


Device set to use cpu



--- Chatbot RAG ---
Tú: hola
--------------------------------------------------
Chatbot RAG: Hola! En que puedo ayudarte?
Similitud: 0.9834
--------------------------------------------------
Tú: los productos incluyen iva
--------------------------------------------------
Chatbot RAG: Vendemos ropa y accesorios para mujer.
Similitud: 0.6899
--------------------------------------------------
Tú: como cancelo una compra 
--------------------------------------------------
Chatbot RAG: Si el pedido aún no fue despachado, podés cancelarlo desde “Mis compras” seleccionando la opción “Cancelar compra”.
Similitud: 0.9255
--------------------------------------------------
Tú: no me llego el seguimiento
--------------------------------------------------
Chatbot RAG: Si no lo recibiste, escribinos por el chat o al mail de atención al cliente y te lo volvemos a enviar.
Similitud: 0.6132
--------------------------------------------------
Tú: como creo una cunta
------------------------------------

### Resultados de la similitud de respuestas del chatobot RAG:

0. 0.98
1. 0.68 (respuesta incorrecta)
2. 0.92
3. 0.61
4. 0.93
5. 0.78

El chatbot supo responder todas las preguntas y se equivoco solo en una.

## Conclusiones

Para este trabajo utilicé como aplicación de chatbot una atención al cliente de una tienda de ropa online.
Implementé el uso de 3 tipos de chatbots, uno con matriz TF-IDF, otro con embeddings usando el modelo BERT, y por último un chatbot RAG usando el modelo 'google/flan-t5-small' con el msmo embedding de BERT.

Hice dos versiones de chatbots TF-IDF y embeddings para asegurar que estaba tomando la decisión correcta con el preprocesamiento de mi dataset. Una versión tiene una limpieza muy básica donde se elimanan solo puntos, comas, comillas, paréntesis, guiones medios, dos puntos, etc. y otra versión tiene una limpieza más avanzada que implica la eliminación de stopwords y una lematización.

El objetivo de estas versiones era constatar de que una limpieza avanzada para un dataset simple como el mio iba a eliminar mucha información a mis preguntas y los chatbot no iban a tener un buen rendimiento. Esto lo confirme con los resultados que obtuve, en los que realice 6 preguntas iguales a cada chatbot y evalué su similitud de respuesta (la primer pregunta es un saludo inicial: 'Hola').

Las preguntas fueron:
0. Hola. Hola! En que puedo ayudarte?
1. ¿Los precios incluyen IVA? Sí, todos los precios publicados ya tienen el IVA incluido.
2. ¿Cómo cancelo una compra? Si el pedido aún no fue despachado, podés cancelarlo desde “Mis compras” seleccionando la opción “Cancelar compra”.
3. No me llegó el seguimiento. Si no lo recibiste, escribinos por el chat o al mail de atención al cliente y te lo volvemos a enviar.
4. ¿Cómo creo una cuenta? Entrá en “Crear cuenta”, completá tus datos y listo. Solo necesitás un mail válido.
5. ¿Hacen envíos? Sí, realizamos envíos a todo el país con Correo Argentino.

Y los resultados fueron:

| Preguntas | TF-IDF | Embeddings | TF-IDF LA | Embedding LA | Chatbot RAG |
|----|--------|------------|-----------|--------------|--------------|
| 0  | 1.0  | 0.98        | 1.0       | 1.0         |0.98
| 1  | 1.0   | -0.55        | -0.57       | 0.73          |/0.68
| 2  | 0.93  | 0.92        | /0.63       | 0.95          |0.92
| 3  | 0.69   | 0.61       | 0.70       | -0.43          |0.61
| 4  | 0.93  | 0.93      | 0.0       | -0.50         |0.93
| 5  | 0.70   | 0.78        | 0.0     | /0.74        |0.78

Los que tienen un - corresponden a respuestas donde el chatbot no supo responder y los que tienen / corresponden a respuestas incorrectas.

La tabla muestra un mejor rendimiento de los chatbots con preprocesamientos simples. Los otros chatbots que tuvieron una limpieza avanzada tienen menor porcetanjes de respuestas correctas y uno mayor de respuestas que no supo comprender. Por lo tanto, reafirmo que para un dataset simple como la antención al cliente de una tienda de ropa no necesita un preprocesamiento complejo.

Por otro lado, en el chatbot RAG utilicé la limpieza simple porque investigué que los modelos para este tipo de aplicaciones suelen contar con un buen entendimiento a textos sin procesar. Su funcionamiento rindió muy parecido al primero modelo de TF-IDF, siendo los mejores.

Como conclusión final, en una implementación de chatbots para asistencia virtual de una tienda online de ropa, con un dataset de preguntas y respuestas con leguaje simple. El mejor modelo que se puede aplicar el TF-IDF con una limpieza básica. El chatbot RAG también es una buena opción pero si requiere un mayor tiempo de programación con ajustes más complejos, el TF.IDF sigue siendo la mjor opción. En cuanto a el error en el tipeo de letras en palabras, los modelos si requiern un mejor ajuste para responder de manera correcta.



---

Por último, en mi experiencia personal fue un trabajo que me costó entender, y además lo hice a contratiempo, por lo que me gustaría haber ordenado mejor el código y agragar más detalles. También tuve mucho uso de las IAs, como gemini, ChatGPT y Copilot porque me encontré en una situación donde tenía que aplicar chatbots en un contexto muy específico y sumado a la falta de tiempo tuve que recurrir especialmente a ellas cuando no encontraba muchas referencias en internet como en otros trabajos, además de los videos y todos los notebooks de las clases.


### Referencias

* Los videos de las clases de la materia
* Los notebook de la materia referidos a limpieza de texto, embedding, chatbots.
* Usos de IAs: Gemini, Copilot, CHATGPT

Modelos utilizados:

* https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
* https://huggingface.co/google/flan-t5-small

Para crear el dataset simplemente hice preguntas y respuestas básicas que puede haber en el contexto de una tienda de ropa online.

